## Toy Examples

In [1]:
# import libraries
import import_ipynb
import toolbox_sccf as sccf
import toolbox_SMC as smc

# other packages
import argparse
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
import time
import numbers
from cvxpy.expressions.expression import Expression
from cvxpy.expressions.constants import Constant

#### `clipped linear regression`

In [2]:
## generate data
np.random.seed(107)
LAMB = .5
eta = .01
N, d = 500, 150
N_outliers = int(N/10+1)
A = np.random.randn(N, d)
x_true = np.random.randn(d)
b = A @ x_true + 2e-1 * np.random.randn(N)
outliers_id = np.random.choice(np.arange(N), replace=False, size=N_outliers)
b[outliers_id] *= -1.

In [3]:
# sccf
x_sccf = cp.Variable(d)
objective_sccf = 0.0
for s in range(N):
    objective_sccf += sccf.minimum(cp.square(A[s]@x_sccf-b[s]), LAMB)
objective_sccf += eta * cp.sum_squares(x_sccf)
prob_sccf = sccf.Problem(objective_sccf)
# smc
x_smc = cp.Variable(d)
objective_smc = smc.minimum(cp.square(A@x_smc-b), LAMB)
objective_smc += eta*cp.sum_squares(x_smc)
prob_smc = smc.Problem(objective_smc)

tic = time.time()
result = prob_sccf.solve(verbose=True)
toc = time.time()
print ("sccf time boyd (s):", toc-tic)
print ("sccf iters boyd:", result["iters"])
x_sccf_val = x_sccf.value
print ('sccf value boyd: '+str(objective_sccf.value))
tic = time.time()
result = prob_smc.solve(method='boyd',verb_=True,extra_verb_=False)
toc = time.time()
print ("smc time boyd (s):", toc-tic)
print ("smc iters boyd:", result["iters"])
x_smc_val_boyd = x_smc.value
print ('smc value boyd: '+str(objective_smc.value))
tic = time.time()
result = prob_smc.solve(method='vandessel',verb_=True,extra_verb_=False)
toc = time.time()
print ("smc time vandessel (s):", toc-tic)
print ("smc iters vandessel:", result["iters"])
x_smc_val_vandessel = x_smc.value
print ('smc value vandessel: '+str(objective_smc.value))

0001 | 2.2587e+02 | 5.0000e-01
0002 | 2.2003e+02 | 2.0000e-01
0003 | 2.1327e+02 | 2.0000e-01
0004 | 1.9928e+02 | 2.0000e-01
0005 | 1.9457e+02 | 2.0000e-01
0006 | 1.9033e+02 | 2.0000e-01
0007 | 1.8966e+02 | 2.0000e-01
0008 | 1.8966e+02 | 2.0000e-01
0009 | 1.8966e+02 | 2.0000e-01
0010 | 1.8966e+02 | 2.0000e-01
0011 | 1.8966e+02 | 0.0000e+00
Terminated (stopping condition satisfied).
sccf time boyd (s): 4.0951361656188965
sccf iters boyd: 11
sccf value boyd: 189.66065700969747
iter. 0001 | Fval. 2.2587e+02 | BICval. 7.8952e+03 | divinf. 2.0000e-01
iter. 0002 | Fval. 2.2003e+02 | BICval. 4.8262e+03 | divinf. 2.0000e-01
iter. 0003 | Fval. 2.1327e+02 | BICval. 1.7527e+03 | divinf. 2.0000e-01
iter. 0004 | Fval. 1.9928e+02 | BICval. 2.1151e+02 | divinf. 2.0000e-01
iter. 0005 | Fval. 1.9457e+02 | BICval. 2.0791e+02 | divinf. 2.0000e-01
iter. 0006 | Fval. 1.9033e+02 | BICval. 2.0386e+02 | divinf. 2.0000e-01
iter. 0007 | Fval. 1.8966e+02 | BICval. 1.9946e+02 | divinf. 2.0000e-01
iter. 0008 | Fval

In [4]:
outliers_sccf = np.where((A@x_sccf_val-b)**2>LAMB)[0]
outliers_smc_boyd = np.where((A@x_smc_val_boyd-b)**2>LAMB)[0]
outliers_smc_vandessel = np.where((A@x_smc_val_vandessel-b)**2>LAMB)[0]

In [5]:
print('undetected outliers for sccf boyd: ' +str(np.setdiff1d(outliers_id,outliers_sccf)))
print('undetected outliers for smc boyd: ' +str(np.setdiff1d(outliers_id,outliers_smc_boyd)))
print('undetected outliers for vandessel: ' +str(np.setdiff1d(outliers_id,outliers_smc_vandessel)))
print('false outliers for sccf boyd: ' +str(np.setdiff1d(outliers_sccf,outliers_id)))
print('false outliers for smc boyd: ' +str(np.setdiff1d(outliers_smc_boyd,outliers_id)))
print('false outliers for vandessel: ' +str(np.setdiff1d(outliers_smc_vandessel,outliers_id)))

undetected outliers for sccf boyd: [121 187 215 286]
undetected outliers for smc boyd: [121 187 215 286]
undetected outliers for vandessel: []
false outliers for sccf boyd: [  0   1   2   6   7  10  11  12  15  16  17  21  22  23  25  26  27  29
  30  32  34  37  38  39  40  41  42  43  45  46  47  49  50  51  52  53
  54  56  57  58  59  61  62  64  66  67  68  69  70  71  74  75  77  78
  79  80  81  82  83  85  87  88  89  92  94  95  96  97  98  99 100 102
 105 106 110 111 112 113 115 116 117 118 119 120 122 123 124 125 126 127
 128 129 130 131 133 134 136 137 138 140 142 144 145 147 148 150 151 152
 155 156 158 160 161 162 163 164 165 167 169 170 171 172 173 174 175 177
 180 181 182 185 186 188 191 192 193 194 195 196 198 199 200 203 206 207
 209 210 211 212 213 214 218 220 221 222 223 225 226 228 229 230 231 232
 236 238 240 242 243 244 246 247 248 250 251 252 253 254 255 257 258 259
 264 265 267 268 269 270 271 272 274 276 277 278 280 282 283 285 287 288
 290 291 292 293 294 295

In [6]:
NREPS = int(30)

reached_val_boyd = []
reached_val_vandessel_ten = []
reached_val_vandessel = []
reached_val_vandessel_tenth = []
reached_val_am = [] 


for rep in range(NREPS):

    ## common init weights ## 
    wsw = prob_smc.weights_setup('random')

    print('new rep #'+str(rep+1))
    prob_smc.solve(method='boyd',verb_=False,extra_verb_=False,warm_start_weights=wsw.copy())
    reached_val_boyd.append(objective_smc.value)
    print ('smc value boyd: '+str(objective_smc.value))
    prob_smc.solve(method='vandessel',schedule_constant=10,verb_=False,extra_verb_=False,warm_start_weights=wsw.copy())
    reached_val_vandessel_ten.append(objective_smc.value)
    print ('smc value vandessel with (SC = 10): '+str(objective_smc.value))
    prob_smc.solve(method='vandessel',schedule_constant=1,verb_=False,extra_verb_=False,warm_start_weights=wsw.copy())
    reached_val_vandessel.append(objective_smc.value)
    print ('smc value vandessel with (SC = 1): '+str(objective_smc.value))
    prob_smc.solve(method='vandessel',schedule_constant=1/10,verb_=False,extra_verb_=False,warm_start_weights=wsw.copy())
    reached_val_vandessel_tenth.append(objective_smc.value)
    print ('smc value vandessel with (SC = 1/10): '+str(objective_smc.value))
    prob_smc.solve(method='am',verb_=False,extra_verb_=False,warm_start_weights=wsw.copy())
    reached_val_am.append(objective_smc.value)
    print ('smc value am: '+str(objective_smc.value))
    print(' ')

new rep #1
smc value boyd: 165.78739891448276
smc value vandessel with (SC = 10): 162.25326308052416
smc value vandessel with (SC = 1): 39.32473412292797
smc value vandessel with (SC = 1/10): 39.32566439954434
smc value am: 208.45233359111594
 
new rep #2
smc value boyd: 156.69155771469715
smc value vandessel with (SC = 10): 159.94676882810322
smc value vandessel with (SC = 1): 39.32473412292794
smc value vandessel with (SC = 1/10): 39.32566439954436
smc value am: 199.63546013057393
 
new rep #3
smc value boyd: 160.27757530076587
smc value vandessel with (SC = 10): 157.59330907402807
smc value vandessel with (SC = 1): 39.324734122927964
smc value vandessel with (SC = 1/10): 39.325664399544344
smc value am: 207.996732366866
 
new rep #4
smc value boyd: 162.6184517145942
smc value vandessel with (SC = 10): 164.64758429100118
smc value vandessel with (SC = 1): 39.324734122927985
smc value vandessel with (SC = 1/10): 39.32566439954436
smc value am: 195.0643938258263
 
new rep #5
smc value 

In [7]:
np.mean(reached_val_boyd) # inexact alternated minimization (comme sccf papier mais en optimisé rapidité) 
                          # + extensible à SMC pour des termes avec min entre + que 2 termes 
    

153.8992834417317

In [8]:
np.mean(reached_val_vandessel) # simulated annealing (normalized) alternated minimization 

39.32473412292796

In [10]:
np.mean(reached_val_vandessel_ten) # schedule annealing 10 x plus aggressif -> ressemble davantage au hard AM

155.84861690002302

In [11]:
np.mean(reached_val_am) # AM (exact alternated minimization)

200.02480458992923